# Preprequisites


Let's install the libraries and import the packages we will need to test this model.

In [1]:
!pip install -U transformers accelerate einops xformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 5.6 MB/s eta 0:00:00


In [2]:
from time import time
from IPython.display import display, Markdown
from torch import cuda, bfloat16
from transformers import BitsAndBytesConfig, AutoConfig, AutoTokenizer, AutoModelForCausalLM

In [3]:
model_id = 'OpenLLM-Ro/RoGemma-7b-Instruct'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
time_1 = time()
model_config = AutoConfig.from_pretrained(
    model_id,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
time_2 = time()
print(f"Download and quantize model: {round(time_2-time_1, 3)} sec.")

config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Download and quantize model: 432.038 sec.


In [5]:
time_1 = time()
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare tokenizer: {round(time_2-time_1, 3)} sec.")

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Prepare tokenizer: 2.797 sec.


# Test the model

Let's quickly test the model with a simple query.

Let's refine a bit the query, let's write a query function.

In [16]:
system_message = "Ești un instructor inteligent care poate să răspundă la întrebări de cultură generală in limba român"

instruction = "Care e cel mai înalt munte din România?"

chat = [
        {"role": "user", "content": instruction},
        ]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, system_message=system_message)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))

<bos><start_of_turn>user
what  is the famous drink in germany ?<end_of_turn>
<start_of_turn>model
În Germania, berea este cea mai populară băutură alcoolică. În timpul sezonului de bere, numit „Mai” sau „Mai”, germanii se adună în parcuri publice și în parcuri pentru a bea bere cu prietenii și familia.<eos>


In [17]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()

    user_message = "Question: " + user_message + " Answer:"
    messages = [
        #{"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
        system_message=system_message
        )
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    inputs = tokenizer.encode(prompt,
                             add_special_tokens=False,
                             return_tensors="pt")

    outputs = model.generate(input_ids=inputs,
                             temperature=temperature,
                             max_new_tokens=max_length)

    answer = tokenizer.decode(outputs[0])
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
Ești un instructor inteligent care poate să răspundă la întrebări de cultură generală in limba română
"""

# Test with the query function

Let's add also a function to beautify the answer.

In [18]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [19]:
user_message = "Care e capitala Romaniei?"
response = query_model(system_message, user_message, 0.1, 32)
display(Markdown(colorize_text(response)))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(




**<font color='red'>Question:</font>** Care e capitala Romaniei? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Care e capitala Romaniei? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Capitala României este București.<eos> 

**<font color='magenta'>Total time:</font>** 1.53 sec.

In [20]:
user_message = "Cine este primarul Bucurestiului?"
response = query_model(system_message, user_message, 0.1, 32)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine este primarul Bucurestiului? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Cine este primarul Bucurestiului? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Primarul orașului București este Nicușor Dan.<eos> 

**<font color='magenta'>Total time:</font>** 2.25 sec.

In [11]:
user_message = "Cine a fost Ion Ghica?"
response = query_model(system_message, user_message, 0.1, 256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine a fost Ion Ghica? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Cine a fost Ion Ghica? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Ion Ghica a fost un om de stat și diplomat român care a servit ca prim-ministru al României în mai multe rânduri între 1852 și 1866. El a jucat un rol esențial în dezvoltarea și modernizarea țării în timpul domniei lui Alexandru al II-lea al Moldovei și a lui Gheorghe al V-lea al Țării Românești. Ghica a fost, de asemenea, implicată în înființarea primului guvern democrat din România și în înființarea primului sistem bancar din țară.<eos> 

**<font color='magenta'>Total time:</font>** 12.37 sec.

In [12]:
user_message = "Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti"
response = query_model(system_message, user_message, 0.1, 128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Listeaza, folosind bullets points, 3 cele mai cunoscute atractii din Bucuresti 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
- Turnul din Catedrala Neapole
- Muzeul Național de Artă al României
- Piața din Piața Unirii<eos> 

**<font color='magenta'>Total time:</font>** 3.86 sec.

In [13]:
user_message = "Ce boarding games pot sa joc cu prietenii diseara?"
response = query_model(system_message, user_message, 0.1, 128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Ce boarding games pot sa joc cu prietenii diseara? 

**<font color='green'>Answer:</font>** <bos><start_of_turn>user


**<font color='red'>Question:</font>** Ce boarding games pot sa joc cu prietenii diseara? 

**<font color='green'>Answer:</font>**<end_of_turn>
<start_of_turn>model
Există mai multe jocuri de masă pe care le puteți juca cu prietenii în seara asta. Unele opțiuni populare includ:

1. Monopoly: Un clasic de bord care implică cumpărarea și dezvoltarea de proprietăți, și încercarea de a ruina afacerile celorlalți jucători.
2. Cluedo: Un joc de detectiv în care jucătorii încearcă să rezolve misterul uciderii unui personaj prin excluderea diferitelor suspiciuni și arme.
3. Catan: Un joc de dezvoltare a resurselor în 

**<font color='magenta'>Total time:</font>** 12.71 sec.